In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import os
import re
import sys
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import yaml
from loguru import logger
from scipy.io import wavfile
from tqdm import tqdm

from enfify.config import ENFIFY_DIR, INTERIM_DATA_DIR, PROCESSED_DATA_DIR
from enfify.pipeline import freq_feature_pipeline

In [ ]:
logger.remove()
logger.add(sys.stderr, level="ERROR")

In [18]:
files_pattern = str(INTERIM_DATA_DIR / "Carioca1" / "*" / "*.wav")
files = sorted(glob(files_pattern))

features_dir = PROCESSED_DATA_DIR / "Carioca1"

os.makedirs(features_dir, exist_ok=True)

for file in tqdm(files):
    basename = os.path.splitext(os.path.basename(file))[0]
    sample_freq, sig = wavfile.read(file)
    with open(ENFIFY_DIR/"config_nature.yml", "r") as f:
        config = yaml.safe_load(f)
    features = freq_feature_pipeline(sig, sample_freq, config)
    np.save(features_dir / f"{basename}.npy", features)

  0%|          | 0/1000 [00:00<?, ?it/s]

2024-08-26 00:45:36.591 | WARNING  | enfify.preprocessing:downsample_scipy:47 - The target sampling rate is not an integer multiple of the current sampling rate. Resampling is used, which does not have an integrated antialiasing filter. Applying a lowpass filter.


  0%|          | 2/1000 [00:00<01:23, 11.96it/s]

2024-08-26 00:45:36.677 | WARNING  | enfify.preprocessing:downsample_scipy:47 - The target sampling rate is not an integer multiple of the current sampling rate. Resampling is used, which does not have an integrated antialiasing filter. Applying a lowpass filter.
2024-08-26 00:45:36.763 | WARNING  | enfify.preprocessing:downsample_scipy:47 - The target sampling rate is not an integer multiple of the current sampling rate. Resampling is used, which does not have an integrated antialiasing filter. Applying a lowpass filter.


  0%|          | 4/1000 [00:00<01:18, 12.77it/s]

...

2024-08-26 00:47:18.634 | WARNING  | enfify.preprocessing:downsample_scipy:47 - The target sampling rate is not an integer multiple of the current sampling rate. Resampling is used, which does not have an integrated antialiasing filter. Applying a lowpass filter.
2024-08-26 00:47:18.746 | WARNING  | enfify.preprocessing:downsample_scipy:47 - The target sampling rate is not an integer multiple of the current sampling rate. Resampling is used, which does not have an integrated antialiasing filter. Applying a lowpass filter.
2024-08-26 00:47:18.780 | WARNING  | enfify.preprocessing:downsample_scipy:47 - The target sampling rate is not an integer multiple of the current sampling rate. Resampling is used, which does not have an integrated antialiasing filter. Applying a lowpass filter.


100%|█████████▉| 997/1000 [01:42<00:00, 11.12it/s]

2024-08-26 00:47:18.883 | WARNING  | enfify.preprocessing:downsample_scipy:47 - The target sampling rate is not an integer multiple of the current sampling rate. Resampling is used, which does not have an integrated antialiasing filter. Applying a lowpass filter.
2024-08-26 00:47:18.986 | WARNING  | enfify.preprocessing:downsample_scipy:47 - The target sampling rate is not an integer multiple of the current sampling rate. Resampling is used, which does not have an integrated antialiasing filter. Applying a lowpass filter.
2024-08-26 00:47:19.074 | WARNING  | enfify.preprocessing:downsample_scipy:47 - The target sampling rate is not an integer multiple of the current sampling rate. Resampling is used, which does not have an integrated antialiasing filter. Applying a lowpass filter.


100%|██████████| 1000/1000 [01:42<00:00,  9.74it/s]

2024-08-26 00:47:19.198 | WARNING  | enfify.preprocessing:downsample_scipy:47 - The target sampling rate is not an integer multiple of the current sampling rate. Resampling is used, which does not have an integrated antialiasing filter. Applying a lowpass filter.
